In [1]:
import numpy as np
from keras.models import load_model
import string
import json
import nltk

Using TensorFlow backend.


In [25]:
def read_glove_vecs(glove_file):
    with open(glove_file,encoding='utf8') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype='float32')
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

def pp(strr):
    words = strr.split()
    stripped = [w.translate(table) for w in words]
    #stripped = list(filter(None, stripped))
    stripped = [word.lower() for word in stripped]
    #stripped = [w for w in stripped if not w in stop_words]
    #stemmed = [porter.stem(word) for word in stripped]
    return(' '.join(stripped))

def sentences_to_indices(X, word_to_index, max_len):

    m = X.shape[0]                                   # number of training examples
    X_indices = np.zeros((m,max_len))
    
    for i in range(m):                               # loop over training examples
        sentence_words = X[i].lower().split()
        j = 0
        
        # Loop over the words of sentence_words
        for w in sentence_words:
            # Set the (i,j)th entry of X_indices to the index of the correct word.
            X_indices[i, j] = word_to_index[w]
            # Increment j to j + 1
            j = j+ 1
            
 
    return X_indices

def cosine_similarity(u, v):
  
    distance = 0.0
    dot = np.dot(u,v)
    norm_u = np.linalg.norm(u)
    
    norm_v = np.linalg.norm(v)
    cosine_similarity = dot/np.multiply(norm_u,norm_v)
    
    return cosine_similarity

def sentence_to_avg(sentence, word_to_vec_map):
    words = sentence.lower().split()
    avg = np.zeros((200,))
    for w in words:
        avg += word_to_vec_map[w]
    avg = avg/len(words)  
    return avg

def batao(labl,sent):
    mini = 0
    for i,n in enumerate(dictt[labl]):
            cs = cosine_similarity(n[0],sentence_to_avg(pp(sent),word_to_vec_map))
            if(cs>mini):
                mini = cs
                resp = n[1]
    return(resp)
def chat2():
    print("start chit chat! (Enter quit to exit)")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break
        x_test = np.array([pp(inp)])
        X_test_indices = sentences_to_indices(x_test, word_to_index, 24)
        pred = model.predict(X_test_indices)
        num = np.argmax(pred)
        lebl = labllist[num]
        print(batao(lebl,inp))

In [3]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('glove6b200d/glove.6B.200d.txt')
model = load_model('chatter.h5')

W1125 03:30:32.744243 17520 deprecation_wrapper.py:119] From D:\MiniConda\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1125 03:30:32.776317 17520 deprecation_wrapper.py:119] From D:\MiniConda\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1125 03:30:32.780783 17520 deprecation_wrapper.py:119] From D:\MiniConda\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1125 03:30:33.453666 17520 deprecation_wrapper.py:119] From D:\MiniConda\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W1125 03:30:33.471426 17520 de

In [26]:
table = str.maketrans('', '', string.punctuation)

In [27]:
with open("intents.json") as file:
    data = json.load(file)

In [28]:
words = []
labels = []
docs_x = []
docs_y = []
xx = []
for intent in data["intents"]:
        for pattern in intent["patterns"]:
            ##stemming to eliminate extra words
            ##tokeninze
            wrds = nltk.word_tokenize(pattern) ##list of words in patters
            words.extend(wrds) ##instead of append
            xx.append(pattern)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])
            
        if intent["tag"] not in labels:
            labels.append(intent["tag"])
            
labels = sorted(labels)      

In [29]:
fasa=[]
for line in xx:
    words = line.split()
    stripped = [w.translate(table) for w in words]
    stripped = [word.lower() for word in stripped]
    fasa.append(' '.join(stripped))
X_train = [i for i in fasa if i]

In [30]:
reps = []
for intent in data["intents"]:
        for responses in intent["responses"]:
            reps.append(responses)

In [31]:
len(reps)==len(X_train)

True

In [32]:
dictt = {}
for i,n in enumerate(X_train):
    if not docs_y[i] in dictt.keys():
        dictt[docs_y[i]]=[]
    dictt[docs_y[i]].append([sentence_to_avg(pp(n),word_to_vec_map),reps[i]])

In [33]:
labllist = np.unique(np.array(docs_y))
print(labllist)

['Admission' 'Courses' 'Fees' 'General' 'Health' 'Other' 'Refunds'
 'goodbye' 'greeting' 'negative' 'positive']


In [34]:
chat2()

start chit chat! (Enter quit to exit)
You: hi
Hello, thanks for visiting
You: fees rates in quebec
Sorry, did I give the wrong answer?
You: who sets fees rates in quebec
The Quebec Ministry of Education, Leisure and Sports sets the base tuition fee rate as well as the rates of premiums, or forfaitaires, charged to non Quebec residents and International students. Additional information is available on the Ministere de lEducation, Loisirs et Sportswebsite.
You: how mch i have to pay
Go to your student portal under the MyConcordia Menu choose "Financial » Students Account". Your student account will indicate the amount due each term and the deadline date by which your payment must appear on your student account.
You: how much i pay
Go to your student portal under the MyConcordia Menu choose "Financial » Students Account". Your student account will indicate the amount due each term and the deadline date by which your payment must appear on your student account.
You: lol
See you later, than